In [1]:
from tkinter import *
from tkinter.ttk import *
from tkinter.filedialog import askdirectory
import os
import numpy as np
from PIL import Image, ImageTk
import cv2
import imutils
from imutils import paths
import threading
from ultralytics import YOLO


# Create the main window
ws = Tk()
ws.title('AI-Enhanced Image Stitching and Edge Detection')
ws.geometry('900x700')

images = [] # List to hold loaded images
stitched_img=[]# Variable to store the stitched image



# function updates the log_label widget with any status or error messages during operations.
def log_message(message):
    log_label.config(text=message)
    ws.update_idletasks()



# function allows the user to select a folder containing images for stitching.
# then it loads and processes the images into an RGB format to images list.
def open_file():
    folder_path = askdirectory(title="Select a Folder")
    if not folder_path:
        Label(ws, text='No Folder Selected', foreground='red').grid(row=4, columnspan=3, pady=30,padx=300)
        return
    
    log_message("loading images...")


    imagePaths = sorted(list(paths.list_images(folder_path)))
    global images 
    images.clear()  # Reset images list if new files are loaded
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert the image to RGB format
            images.append(image)

    # Check if images were successfully loaded
    if not images:
        Label(ws, text='No valid images loaded', foreground='red').grid(row=4, columnspan=3, pady=30,padx=300)
        return


# Function to handle file upload and initiate the process
def uploadFiles():
    open_file() 
    upld0.grid_forget()  
    pb1 = Progressbar(ws, orient=HORIZONTAL, length=300, mode='determinate')
    pb1.grid(row=8, columnspan=3, pady=30,padx=300)

    # Progress bar to indicate image upload
    def progress(step):
        if step <= 5:
            pb1['value'] += 20
            ws.after(1000, progress, step+1)
        else:
            pb1.destroy()
            Label(ws, text=f'All Files Uploaded Successfully! ({len(images)}/{len(images)})', foreground='green').grid(row=0, columnspan=4, pady=30,padx=300)
            show_images()
           
    threading.Thread(target=progress, args=(1,)).start()



# Function function allows the user to preview the loaded images before stitching in a scrollable frame horizontally
def show_images():
    global images
    if images:
        # Create a new frame for displaying images
        image_frame = Frame(ws)
        image_frame.grid(row=2, column=0, columnspan=2,pady=5,padx=30)
        

        canvas = Canvas(image_frame, height=140, width=800)  
        canvas.pack(side=LEFT, fill=BOTH, expand=True)
        

        # Add a scrollbar for horizontal scrolling
        scrollbar = Scrollbar(image_frame, orient=HORIZONTAL, command=canvas.xview)
        scrollbar.pack(side=BOTTOM, fill=X,)
       

        canvas.configure(xscrollcommand=scrollbar.set)

       # Create a container for images inside the canvas
        image_container = Frame(canvas)
        canvas.create_window((0, 0), window=image_container, anchor='nw')

        for i, img in enumerate(images):
            img_pil = Image.fromarray(img)  # Convert OpenCV image to PIL
            img_pil = img_pil.resize((100, 100))  # Resize to fit in the UI
            img_tk = ImageTk.PhotoImage(img_pil)  # Convert image for Tkinter
            
            label = Label(image_container, image=img_tk)
            label.image = img_tk  # Keep a reference to avoid garbage collection
            label.grid(row=0, column=i, pady=10,padx=20)

        # Adjust the scroll region to encompass the entire image sequence
        image_container.update_idletasks()
        canvas.config(scrollregion=canvas.bbox(ALL))

        # Start stitching images in a separate thread
        threading.Thread(target=stitcher).start()


# Function to stitch images together and crops the resulting image to remove unnecessary borders.
def stitcher():
    global images
    global stitched_img
    if not images:
        Label(ws, text="No images to stitch", foreground='red').grid(row=7, column=5, pady=30, padx=300)
        return
    
    log_message("stitching images in progress...")

    # Resize images to a common size for stitching
    resized_images = [cv2.resize(image, (1024, 787)) for image in images]
    
    stitcher = cv2.Stitcher_create() if cv2.__version__.startswith('4') else cv2.createStitcher()
    (status, stitched) = stitcher.stitch(resized_images)

    # Check if stitching was successful
    if status == 0:
        log_message("stitching successful, applying cropping...")


        stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,cv2.BORDER_CONSTANT, (0, 0, 0))

        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        # find all external contours in the threshold image then find
		# the *largest* contour which will be the contour/outline of
		# the stitched image 
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
		# allocate memory for the mask which will contain the
		# rectangular bounding box of the stitched image region
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

        # create two copies of the mask: one to serve as our actual
		# minimum rectangular region and another to serve as a counter
		# for how many pixels need to be removed to form the minimum
		# rectangular region
        minRect = mask.copy()
        sub = mask.copy()
		# keep looping until there are no non-zero pixels left in the
		# subtracted image
        while cv2.countNonZero(sub) > 0:
			# erode the minimum rectangular mask and then subtract
			# the thresholded image from the minimum rectangular mask
			# so we can count if there are any non-zero pixels left
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)

        # find contours in the minimum rectangular mask and then
		# extract the bounding box (x, y)-coordinates
        cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
			cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
		# use the bounding box coordinates to extract the our final
		# stitched image
        stitched = stitched[y:y + h, x:x + w]
        
    
        # Display the cropped stitched image in a smaller frame within the main window
        stitched_img = Image.fromarray(stitched)
        stitched_img = stitched_img.resize((800, 300))  # Resize for display purposes
        stitched_img_tk = ImageTk.PhotoImage(stitched_img)

        # Create a canvas to display the stitched image
        canvas_stitched = Canvas(ws, width=800, height=300)
        canvas_stitched.grid(row=4, column=0, pady=5, padx=30, columnspan=2)
        canvas_stitched.create_image(0, 0, anchor=NW, image=stitched_img_tk)
        canvas_stitched.image = stitched_img_tk  # Keep a reference to avoid garbage collection

        log_message("Finished cropping...")

    else:
        # If stitching failed, show the error code
        Label(ws, text=f"Stitching failed with error code {status}", foreground='red').grid(row=7, column=5, pady=30, padx=300)
        log_message(f"Stitching failed with status code: {status}")

    # Buttons to apply Canny Edge Detection and AI-based Human Detection
    canny_button = Button(ws, text="Apply Canny Edge Detection and Difference of Gaussians (DoG) edge detection", command=open_canny_window)
    canny_button.grid(row=5, column=0, columnspan=2, pady=5, padx=30)

    Human_Detection_button = Button(ws, text="Apply AI-based Human Detection", command=open_ai_human_detection)
    Human_Detection_button.grid(row=6, column=0, columnspan=2, pady=5, padx=30)


# Function to open a new window for Canny Edge Detection and Difference of Gaussians (DoG) edge detection
def open_canny_window():
    global stitched_img
    canny_window = Toplevel(ws)
    canny_window.title("Canny Edge Detection and Difference of Gaussians (DoG) edge detection")
    canny_window.geometry("1000x400")  

    stitched_img = np.array(stitched_img)

    # Convert stitched image to grayscale and Apply Canny edge detection
    stitched_gray = cv2.cvtColor(stitched_img, cv2.COLOR_RGB2GRAY)
    stitched_gray = cv2.GaussianBlur(stitched_gray, (7, 7), 1)
    median = np.median(stitched_gray)
    lower_threshold = int(max(0, (1 - 0.33) * median))
    upper_threshold = int(min(255, (1 + 0.33) * median))
    print(median, lower_threshold, upper_threshold)

    # Apply automatic Canny edge detection using the computed thresholds
    canny_i = cv2.Canny(stitched_gray, lower_threshold, upper_threshold)

    img_dilation = cv2.dilate(canny_i, (7, 7), iterations=1)

    # Convert edges to ImageTk for display
    edges_img = Image.fromarray(img_dilation)
    edges_img = edges_img.resize((800, 300))  # Resize for display
    edges_img_tk = ImageTk.PhotoImage(edges_img)

    # Display Canny edge image in the new window
    canvas_canny = Canvas(canny_window, width=800, height=300)
    canvas_canny.grid(row=0, column=0, padx=20, pady=20)
    canvas_canny.create_image(0, 0, anchor=NW, image=edges_img_tk)
    canvas_canny.image = edges_img_tk

    #-----------------------------------------------------------

    # Apply Gaussian Blur with two different kernels
    blur1 = cv2.GaussianBlur(stitched_gray, (7, 7), 1)
    blur2 = cv2.GaussianBlur(stitched_gray, (19, 19), 3)

    # Compute Difference of Gaussians
    dog = cv2.absdiff(blur1, blur2)

    # Create a canvas to display DoG edge detection
    canvas_dog = Canvas(canny_window, width=800, height=300)
    canvas_dog.grid(row=3, column=0, padx=20, pady=20)

    # Display the initial DoG result without morphology
    dog_img = Image.fromarray(dog)
    dog_img = dog_img.resize((800, 300))  # Resize for display
    dog_img_tk = ImageTk.PhotoImage(dog_img)
    canvas_dog.create_image(0, 0, anchor=NW, image=dog_img_tk)
    canvas_dog.image = dog_img_tk  # Keep reference to avoid garbage collection

    # Function to update morphological kernel size and shape based on slider and radio button selection
    def apply_morphology(val=None):
        kernel_size = int(kernel_slider.get())  # Get kernel size from slider
        if kernel_size < 1:
            kernel_size = 1  # Ensure kernel size is at least 1

        # Get the selected kernel shape
        shape = kernel_shape.get()
        
        if shape == "a rectangular":
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        elif shape == "an elliptic":
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        else:
            kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (kernel_size, kernel_size))

        # Apply morphology operation (closing) with the selected kernel
        morphed = cv2.morphologyEx(dog, cv2.MORPH_CLOSE, kernel)

        # Convert morphed result to ImageTk for display
        morphed_img = Image.fromarray(morphed)
        morphed_img = morphed_img.resize((800, 300))  # Resize for display
        morphed_img_tk = ImageTk.PhotoImage(morphed_img)

        # Update canvas with the new morphed image
        canvas_dog.create_image(0, 0, anchor=NW, image=morphed_img_tk)
        canvas_dog.image = morphed_img_tk  # Keep reference to avoid garbage collection

    # Add radio buttons to select morphological kernel shape
    kernel_shape = StringVar(value='a rectangular')  # Set default shape
    shapes = ('a rectangular', 'an elliptic', 'a cross')
    for i in range(len(shapes)):
        r = Radiobutton(canny_window, text=shapes[i], value=shapes[i], variable=kernel_shape, command=apply_morphology)
        r.grid(row=3, column=i+2, padx=5, pady=5)

    # Add slider to control morphological operation kernel size (next to the DoG image)
    kernel_slider = Scale(canny_window, from_=1, to=30, orient=VERTICAL, command=apply_morphology)
    kernel_slider.set(1)  # Set initial value
    kernel_slider.grid(row=3, column=1, padx=5, pady=5)  # Position slider next to the DoG image

    kernel_label = Label(canny_window, text=" Change Morphological kernel size and shape:",foreground='blue')
    kernel_label.grid(row=2, column=3, padx=5, pady=5)  # Position label next to the kernel slider



# function applies a pre-trained YOLO model to detect humans in the stitched image.
def open_ai_human_detection():
    global stitched_img  

    # Create new window for AI-Human Detection
    ai_human_window = Toplevel(ws)
    ai_human_window.title("AI-Human Detection")
    ai_human_window.geometry("1000x400")  # Set window size
 


    # Load the YOLOv5s model (pre-trained)
    model = YOLO('yolov8n.pt')

    # Run detection on the stitched image
    results = model(stitched_img)

    # Filter results to keep only humans (class ID == 0) and confidence > 0.5
    human_results = []
    for result in results:
        for detection in result.boxes:
            if detection.cls == 0 and detection.conf > 0.5:  
                human_results.append(detection)

    # If no humans detected, return a massege
    if not human_results:
        log_label = Label(ai_human_window, text="No humans detected with confidence above 50%", foreground='red')
        log_label.grid(row=0, column=0, columnspan=3, pady=30,padx=300)
        return

    # Plot only the human detections on the image
    result_img = stitched_img.copy()  # Copy the image to draw on
    result_img =np.array(result_img)
    for detection in human_results:
        x1, y1, x2, y2 = map(int, detection.xyxy[0].tolist())  # Get bounding box coordinates
        confidence = detection.conf.item()  # Confidence score as a float
        cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw a green rectangle around the human

        # Prepare the confidence text
        confidence_text = f"{confidence:.2f}"  # Format confidence as a percentage (e.g., 0.85 -> '85%')
        
        # Put the confidence text on the image near the top-left corner of the bounding box
        cv2.putText(result_img, confidence_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Convert the image to the correct format for Tkinter
    result_img_pil = Image.fromarray(result_img)
    result_img_pil = result_img_pil.resize((800, 300))  # Resize the image for display

    # Convert the image to ImageTk format for Tkinter
    result_img_tk = ImageTk.PhotoImage(result_img_pil)

    # Create a canvas in the AI-Human Detection window to display the image
    canvas = Canvas(ai_human_window, width=800, height=300)
    canvas.grid(row=0, column=0, padx=20, pady=20)
    
    # Add the image to the canvas
    canvas.create_image(0, 0, anchor=NW, image=result_img_tk)
    canvas.image = result_img_tk  



# Log label for displaying [INFO] messages
log_label = Label(ws, text="", foreground='green')
log_label.grid(row=1, pady=10, padx=300)

# Button to trigger image upload and processing
upld0 = Button(ws, text='Upload folder containing images for stitching', command=uploadFiles)
upld0.grid(row=5, pady=30, padx=300)


ws.mainloop()
